<a href="https://colab.research.google.com/github/visumania/TFM-AMM/blob/main/cuadernos/entrenaBertGenericoBueno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 5.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers
import tensorflow as tf
import torch
import numpy as np
import imblearn
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import ClusterCentroids
from collections import Counter
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SVMSMOTE
from keras.preprocessing.sequence import pad_sequences# Set the maximum sequence length.
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer, AutoModelForSequenceClassification, AutoTokenizer
#from transformers import * # Load the BERT tokenizer.
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import re

import nlpaug.augmenter.word as naw

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

import pickle


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [14]:
etiqueta = 'IDEOLOGICAL-INEQUALITY'
# etiqueta = 'STEREOTYPING-DOMINANCE'
# etiqueta = 'OBJECTIFICATION'
# etiqueta = 'SEXUAL-VIOLENCE'
# etiqueta = 'MISOGYNY-NON-SEXUAL-VIOLENCE'

In [15]:
MAX_LEN = 120

#df = pd.read_csv('/content/drive/MyDrive/I2C/trainEnFinal.csv', sep=',')
df = pd.read_csv('/content/drive/MyDrive/I2C/Adrián Moreno/EXIST2024/Datasets/training/EXIST2024_training_formated.csv')

df = df.dropna()

df

,id_EXIST,text,IDEOLOGICAL-INEQUALITY,STEREOTYPING-DOMINANCE,OBJECTIFICATION,SEXUAL-VIOLENCE,MISOGYNY-NON-SEXUAL-VIOLENCE
0,110001,2+2=5 MITO Albert Einstein tenía bajo rendimie...,1,0,0,0,0
1,110002,CUANDO UNA MUJER VA A LUCHAR POR SUS DERECHOS,1,0,0,0,1
2,110003,ІЯ ЕГЕЯ Е MOA ¿El Partido Republicano busca pe...,0,1,0,0,0
3,110004,"Paises que ""apoyan"" los derechos de la mujer A...",1,0,0,0,0
4,110005,Ya verás como este 8 de marzo hay uno que te s...,1,0,0,0,0
...,...,...,...,...,...,...,...
4039,212006,u gon act like a bitch u gon die like a bitch,0,0,0,1,1
4040,212007,SHE LOOKS LIKE EVERY OTHER BITCH LIKE makeamem...,0,0,1,0,1
4041,212008,YOURE A BASIC BITCH CASE DISMISSED,0,1,1,0,0
4042,212009,WHEN YOU'RE AUNT HAS THIS WEIRD ASS MAN AND SH...,0,0,0,0,0


In [18]:
def crearinputs(text,tokenizer):
   sentences = text
   input_ids = []# For every sentence...
   for sent in sentences:

         encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
         input_ids.append(encoded_sent)# Print sentence 0, now as a list of IDs.
   #print('Max sentence length: ', max([len(sen) for sen in input_ids]))
   #print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
   #print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_tokid))# Pad our input tokens with value 0.
   input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
   #print('\InputIDS Done.')
   return (input_ids)

In [17]:
def crearmask(input_ids):
   # Create attention masks
   attention_masks = []# For each sentence...
   for sent in input_ids:
       att_mask = [int(tokid > 0) for tokid in sent]
       attention_masks.append(att_mask)
   #print('\Attention masks Done.')
   return(attention_masks)

In [16]:
def crearDataLoad(df,dfv,tokenizer):
 dft=dfv  # usamos el del validacion para que nos salga los resultados y no tener que cambiar mucho codigo
 #sentencias=df['tweet']
 sentencias=df['text'].str.lower()  # para modelos uncased
 labels=df[etiqueta]
 #test=dfv['tweet']
 test=dft['text'].str.lower() # para modelos uncased   ### el fichero de validacion es el que se usa para obtener el resultado final
 ls=dft[etiqueta]
 #validation=dfv['tweet']
 validation=dfv['text'].str.lower()  # para modelos uncased  # el fichero que hemos llamado test es usado en la red neuronal
 validation_labels=dfv[etiqueta]
 train=sentencias
 train_labels=labels
 #print("tamaño del train")
 #print(len(train))
 #print("tamaño de la validacion")
 #print(len(validation))
 #print("tamaño del test")
 #print(len(test))
 #print('Loading tokenizer...')
 train_inputs = crearinputs (train,tokenizer)
 #train_inputs, train_labels = balancea (train_inputs, train_labels)
 train_masks= crearmask (train_inputs)
 test_inputs = crearinputs (test,tokenizer)
 test_masks= crearmask (test_inputs)
 validation_inputs = crearinputs (validation,tokenizer)
 validation_masks= crearmask (validation_inputs)
 #print ("distribucion TRAINING antes del resambplin {}".format(Counter(labels)))
 #print ("distribucion TRAINING despues del resambplin {}".format(Counter(train_labels)))
 #print('numero de sentencias TRAINING despues del resampling: ',train_inputs.shape[0])
 #print ("Distribution del test {}".format(Counter(test_labels)))
 #print('numero de sentencias del test: ',test_inputs.shape[0])
 #print ("Distribution del validacion {}".format(Counter(validation_labels)))
 #print('numero de sentencias del validacion: ',validation_inputs.shape[0])
 train_inputs = torch.tensor(train_inputs)
 validation_inputs = torch.tensor(validation_inputs)
 train_labels = torch.tensor(train_labels.values)
 #train_labels = torch.tensor(train_labels)
 validation_labels = torch.tensor(validation_labels.values)
 #validation_labels = torch.tensor(validation_labels)
 train_masks = torch.tensor(train_masks)
 validation_masks = torch.tensor(validation_masks)
 #test_masks= torch.tensor(test_masks)
 #test_inputs= torch.tensor(test_inputs)
 #test_labels= torch.tensor(test_labels.values)
 #test_labels= torch.tensor(test_labels)
 from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler# The DataLoader needs to know our batch size for training, so we specify it
 batch_size= 32
 train_data = TensorDataset(train_inputs, train_masks, train_labels)
 train_sampler = SequentialSampler(train_data)
 train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)# Create the DataLoader for our validation set.
 validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
 validation_sampler = SequentialSampler(validation_data)
 validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
 #test_data = TensorDataset(test_inputs, test_masks, test_labels)
 #test_sampler = SequentialSampler(test_data)
 #test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)# Create the DataLoader for our validation set.
 return train_dataloader,validation_dataloader #,test_dataloader


In [19]:
def training_model(train_dataloader,validation_dataloader,model,tokenizer):

 model.cuda()
 from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler# The DataLoader needs to know our batch size for training, so we specify it
 batch_size= 32
 # Note: AdamW is a class from the huggingface library (as opposed to pytorch)
 # I believe the 'W' stands for 'Weight Decay fix"
 optimizer = AdamW(model.parameters(),
   lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
   eps = 1e-8 # args.adam_epsilon - default is 1e-8.
   )
 from transformers import get_linear_schedule_with_warmup# Number of training epochs (authors recommend between 2 and 4)
 epochs = 5
 # Total number of training steps is number of batches * number of epochs.
 total_steps = len(train_dataloader) * epochs# Create the learning rate scheduler.
 #print ('total steps', total_steps)
 scheduler = get_linear_schedule_with_warmup(optimizer,
      num_warmup_steps = 0, # Default value in run_glue.py
      num_training_steps = total_steps)

 import numpy as np# Function to calculate the accuracy of our predictions vs labels

 def flat_accuracy(preds, labels):
   pred_flat = np.argmax(preds, axis=1).flatten()
   labels_flat = labels.flatten()
   return np.sum(pred_flat == labels_flat) / len(labels_flat)

 import time
 import datetime
 def format_time(elapsed):
    # Takes a time in seconds and returns a string hh:mm:ss
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


 import random# This training code is based on the `run_glue.py` script here:
 # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
 # Set the seed value all over the place to make this reproducible.
 seed_val = 42
 random.seed(seed_val)
 np.random.seed(seed_val)
 torch.manual_seed(seed_val)
 torch.cuda.manual_seed_all(seed_val)# Store the average loss after each epoch so we can plot them.

 loss_values = []# For each epoch...
 acc_values =  []

 def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label



 for epoch_i in range(0, epochs):

   # ========================================
   # Training
   # ========================================


   print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
   print('Training...') # Measure how long the training epoch takes.
   t0 = time.time() # Reset the total loss for this epoch.
   total_loss = 0 # Put the model into training mode. Don't be mislead--the call to
   total_train_acc = 0
   model.train() # For each batch of training data...
   for step, batch in enumerate(train_dataloader): # Progress update every 40 batches.
     if step % 10 == 0 and not step == 0:
          elapsed = format_time(time.time() - t0)
          print(' Batch {:>5,} of {:>5,}. Elapsed: {:}.'.format(step, len(train_dataloader), elapsed)) # Unpack this training batch from our dataloader.
     b_input_ids = batch[0].to(device)
     b_input_mask = batch[1].to(device)
     b_labels = batch[2].to(device) # Always clear any previously calculated gradients before performing a
     model.zero_grad()  # Perform a forward pass (evaluate the model on this training batch).
     outputs = model(b_input_ids,  attention_mask=b_input_mask,  labels=b_labels) #toktype_ids=None,

     loss = outputs[0] # Accumulate the training loss over all of the batches so that we can
     total_loss += loss.item() # Perform a backward pass to calculate the gradients.
     loss.backward() # Clip the norm of the gradients to 1.0.


     torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Update parameters and take a step using the computed gradient.

     optimizer.step() # Update the learning rate.
     scheduler.step() # Calculate the average loss over the training data.
   avg_train_loss = total_loss / len(train_dataloader)
   loss_values.append(avg_train_loss)

   print(" Average training loss: {0:.2f}".format(avg_train_loss))
   print(" Training epoch took: {:}".format(format_time(time.time() - t0)))
   # ========================================
   # (validation)
   # ========================================
   print("Running validation...")
   t0 = time.time() # Put the model in evaluation mode--the dropout layers behave differently
   # during evaluation.
   model.eval() # Tracking variables
   eval_loss, eval_accuracy = 0, 0
   nb_eval_steps, nb_eval_examples = 0, 0 # Evaluate data for one epoch
   for batch in validation_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch
      with torch.no_grad():  # Forward pass, calculate logit predictions.
          outputs = model(b_input_ids,
                  #toktype_ids=None, #
                  attention_mask=b_input_mask)
      logits = outputs[0] # Move logits and labels to CPU
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      eval_accuracy += tmp_eval_accuracy # Track the number of batches
      nb_eval_steps += 1 # Report the final accuracy for this validation run.
   print(" Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
   # Tracking variables test
   print(" Validation took: {:}".format(format_time(time.time() - t0)))
 print("")
 print("Training complete!")
 #torch.save(model.state_dict(),'/content/drive/MyDrive/I2C/Modelos/modeloValidacion'+etiqueta)
 print ('modelo grabado')
 return model
###############################################

In [20]:
def resultados(validation_dataloader, model, tokenizer):

  model.eval()

  # Tracking variables
  predictions , true_labels = [], []
  pred = []
  t_label =[]
  # Predict
  for batch in validation_dataloader:
    # Add batch to GPU , como no tengo lo dejo aquí
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, #toktype_ids=None, #
                      attention_mask=b_input_mask)

    logits = outputs[0]


    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

    for l in logits:
      # para cada tupla del logits, se selecciona 0 o 1 dependiendo del valor
      # que sea el mayor (argmax)
      pred_labels_i = np.argmax(l).item()
      pred.append(pred_labels_i)

  pred=np.asarray(pred).tolist()
  t_label = [val for sublist in true_labels for val in sublist] # para aplanar la lista de etiquetas
  #print('predicciones',pred)
  #print('t_labels',t_label)
  #print('validation_labels',validation_labels )
  print("RESULTADOS KFOLD validacion cruzada")
  from sklearn.metrics import confusion_matrix
  from sklearn.metrics import classification_report
  print(classification_report(t_label, pred))
  print ("Distribution test {}".format(Counter(t_label)))
  from sklearn.metrics import confusion_matrix
  print(confusion_matrix(t_label, pred))
  from sklearn.metrics import roc_auc_score
  print('AUC ROC:')
  print(roc_auc_score(t_label, pred))
  from sklearn.metrics import f1_score
  result=f1_score(t_label, pred, average='macro',labels=[0,1],pos_label=1,zero_division=0)
  print('f1-score macro:')
  print(result)
  print("****************************************************************")
  return result

**Ejercutar esta celda para validación cruzada⏪**

In [21]:
##################### programa principal #######################################
##########################################################
##########################################################
##########################################################
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('no hay gpu')
print('Found GPU at: {}'.format(device_name))

#import torch# If there's a GPU available...
if torch.cuda.is_available():  # Tell PyTorch to use the GPU.
 device = torch.device("cuda")
 print('There are %d GPU(s) available.' % torch.cuda.device_count())
 print('We will use the GPU:', torch.cuda.get_device_name(0))# If not...
else:
 print('No GPU available, using the CPU instead.')
 device = torch.device("cpu")




#### preparando validación cruzada #######
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
train_data = pd.DataFrame(df)
y=pd.DataFrame(df[etiqueta])
n=5
kf = StratifiedKFold(n_splits=n, random_state=1, shuffle=True)
results = []
print ("COMENZANDO PROCESO DE VALIDACION CRUZADA")
for train_index, val_index in kf.split(train_data,y):

    print("Entrenando KFoldStratificada ")
    print("TRAIN:", train_index, "TEST:", val_index)
    # splitting Dataframe (dataset not included)
    train_df = train_data.iloc[train_index]
    val_df = train_data.iloc[val_index]
    modelB =  BertForSequenceClassification.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    #tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
    #modelB = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

    train_dataloader,validation_dataloader = crearDataLoad(train_df,val_df,tokenizer)
    model=training_model(train_dataloader,validation_dataloader, modelB,tokenizer) # el modelo se graba en modeloValidacion
  # obteniendo metricas del modelo generado en el paso anterior
    model.eval()
    result= resultados(validation_dataloader, model,tokenizer)
    # append model score
    results.append(result)


print("results",results)
print(f"Mean-Precision: {sum(results) / len(results)}")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4
COMENZANDO PROCESO DE VALIDACION CRUZADA
Entrenando KFoldStratificada 
TRAIN: [   0    1    2 ... 4041 4042 4043] TEST: [   8   15   19   25   28   34   39   40   42   45   46   49   54   56
   70   83   87   90   92   95   98  106  107  114  119  121  123  134
  135  137  145  149  154  156  163  166  167  171  173  183  186  191
  196  199  200  228  235  254  255  258  261  262  273  282  284  294
  301  308  312  318  322  328  330  333  334  336  342  343  351  352
  363  369  388  393  395  403  406  408  411  412  415  420  424  430
  434  441  450  454  457  460  468  471  474  483  489  499  500  507
  515  516  520  526  529  534  539  542  543  544  551  557  559  574
  580  581  584  588  590  593  598  599  602  604  606  607  609  616
  624  625  626  627  628  632  646  649  650  657  662  667  672  680
  686  693  698  700  720  724  727  735  740  744  747  750  752  755
  757  780 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


======== Epoch 1 / 5 ========
Training...
 Batch    10 of   102. Elapsed: 0:00:07.


KeyboardInterrupt: 

Ejecutar esta celda para entrenamiento normal (20% validación)

In [22]:
train_data = pd.DataFrame(df)

train_df, validation_df = train_test_split(train_data,random_state=2018, test_size=0.2, stratify=train_data[etiqueta])

##################### programa principal #######################################
##########################################################
##########################################################
##########################################################
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('no hay gpu')
print('Found GPU at: {}'.format(device_name))

#import torch# If there's a GPU available...
if torch.cuda.is_available():  # Tell PyTorch to use the GPU.
 device = torch.device("cuda")
 print('There are %d GPU(s) available.' % torch.cuda.device_count())
 print('We will use the GPU:', torch.cuda.get_device_name(0))# If not...
else:
 print('No GPU available, using the CPU instead.')
 device = torch.device("cpu")


modelB =  BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
#modelB = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

train_dataloader,validation_dataloader = crearDataLoad(train_df, validation_df,tokenizer)
model=training_model(train_dataloader,validation_dataloader, modelB,tokenizer) # el modelo se graba en modeloValidacion
#torch.save(model.state_dict(),'./prueba2')
# obteniendo metricas del modelo generado en el paso anterior
model.eval()
result = resultados(validation_dataloader, model,tokenizer)
# append model score

### para leer el modelo creado habría que poner algo así
###
#model_state_dict = torch.load("/content/drive/MyDrive/ProfNer/modeloEpoch5/profner",  map_location='cpu')
#model = BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path="dccuchile/bert-base-spanish-wwm-cased", state_dict=model_state_dict, cache_dir='./data')


Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


======== Epoch 1 / 5 ========
Training...
 Batch    10 of   102. Elapsed: 0:00:05.
 Batch    20 of   102. Elapsed: 0:00:11.
 Batch    30 of   102. Elapsed: 0:00:17.
 Batch    40 of   102. Elapsed: 0:00:22.
 Batch    50 of   102. Elapsed: 0:00:28.
 Batch    60 of   102. Elapsed: 0:00:34.
 Batch    70 of   102. Elapsed: 0:00:39.
 Batch    80 of   102. Elapsed: 0:00:45.
 Batch    90 of   102. Elapsed: 0:00:51.
 Batch   100 of   102. Elapsed: 0:00:57.
 Average training loss: 0.55
 Training epoch took: 0:00:58
Running validation...
 Accuracy: 0.79
 Validation took: 0:00:05
======== Epoch 2 / 5 ========
Training...
 Batch    10 of   102. Elapsed: 0:00:06.
 Batch    20 of   102. Elapsed: 0:00:12.
 Batch    30 of   102. Elapsed: 0:00:18.
 Batch    40 of   102. Elapsed: 0:00:23.
 Batch    50 of   102. Elapsed: 0:00:29.
 Batch    60 of   102. Elapsed: 0:00:35.
 Batch    70 of   102. Elapsed: 0:00:41.
 Batch    80 of   102. Elapsed: 0:00:47.
 Batch    90 of   102. Elapsed: 0:00:52.
 Batch   100 o

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/I2C/taskB.En.input.csv', sep=',')

#df_test = df_test.rename(columns={"text":"tweet"})

**Cargamos el modelo**

In [ ]:
### para leer el modelo creado habría que poner algo así
###
model_state_dict = torch.load("/content/drive/MyDrive/I2C/Modelos/modeloValidacionrhetorical_question",  map_location='cpu')
model = BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path="bert-base-uncased", state_dict=model_state_dict, cache_dir='./data')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

In [23]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

**Validación del modelo**

In [25]:
def crearDataLoadTrial(dfv,tokenizer): #Ponemos df = None por defecto para que podamos usar dataloads de solo los datos de validación (o entrenamiento)

  print("UN SOLO ARGUMENTO")
  dft=dfv  # usamos el del validacion para que nos salga los resultados y no tener que cambiar mucho codigo

  #validation=dfv['text']
  validation=dfv['text'].str.lower()  # para modelos uncased  # el fichero que hemos llamado test es usado en la red neuronal
  #validation_labels=dfv['label']

  validation_inputs = crearinputs (validation,tokenizer)
  validation_masks= crearmask (validation_inputs)

  validation_inputs = torch.tensor(validation_inputs)

  #validation_labels = torch.tensor(validation_labels.values)


  validation_masks = torch.tensor(validation_masks)

  from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler# The DataLoader needs to know our batch size for training, so we specify it

  #Colab
  batch_size = 32
  #local
  #batch_size = 15

  validation_data = TensorDataset(validation_inputs, validation_masks)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  return validation_dataloader

In [26]:
def prediccion(test_dataloader, model, tokenizer):

  model.eval()

  # Tracking variables
  predictions , true_labels = [], []
  pred = []
  t_label =[]
  # Predict
  for batch in test_dataloader:
    # Add batch to GPU , como no tengo lo dejo aquí
    print('Entro')
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, #toktype_ids=None, #
                      attention_mask=b_input_mask) #Esto falla

    logits = outputs[0] #CREO QUE AQUÍ HABRÍA QUE QUITAR EL 0


    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()


    # Store predictions and true labels
    # Store predictions and true labels
    predictions.append(logits)


    for l in logits:
      # para cada tupla del logits, se selecciona 0 o 1 dependiendo del valor
      # que sea el mayor (argmax)
      pred_labels_i = np.argmax(l).item()
      print(pred_labels_i)
      pred.append(pred_labels_i)

  #Si no me equivoco, en pred guardamos las predicciones hechas por el modelo
  pred=np.asarray(pred).tolist()
  print('predicciones',pred)
  #print('t_labels',t_label)
  #print('validation_labels',validation_labels )

  return pred

In [27]:
device = torch.device("cpu")
#Probamos la solucion que nos han dicho en stackOverflow
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

validation_dataloader = crearDataLoadTrial(df_test,tokenizer)
# obteniendo metricas del modelo generado en el paso anterior
model.eval()
result= prediccion(validation_dataloader, model,tokenizer)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

NameError: name 'df_test' is not defined

In [ ]:
prediccionesSarcasmo = result
print(len(prediccionesSarcasmo))

1400


In [ ]:
prediccionesIronia = result
print(len(prediccionesIronia))

1400


In [ ]:
prediccionesSatira = result
print(len(prediccionesSatira))

1400


In [ ]:
prediccionesUnderstatement = result
print(len(prediccionesUnderstatement))

1400


In [ ]:
prediccionesOverstatement = result
print(len(prediccionesOverstatement))

1400


In [ ]:
arr2d = np.zeros((1400,6))

for i in range(arr2d.shape[0]):
  arr2d[i][0]=prediccionesSarcasmo[i]
  arr2d[i][1]=prediccionesIronia[i]
  arr2d[i][2]=prediccionesSatira[i]
  arr2d[i][3]=prediccionesUnderstatement[i]
  arr2d[i][4]=prediccionesOverstatement[i]


In [ ]:
arr2d

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [ ]:
np.savetxt("/content/drive/MyDrive/I2C/entrega3.txt",arr2d, delimiter=",", header= "sarcasm,irony,satire,understatement,overstatement,rhetorical_question", fmt='%1.0f')

In [ ]:
np.savetxt("/content/drive/MyDrive/I2C/entrega31.txt",result, delimiter=",", header= "rhetorical_question", fmt='%1.0f')